In [1]:
import pandas as pd
import pyarrow.parquet as pq
from datetime import datetime

# 读取数据
price_vol_df = pd.read_parquet(r"D:\workspace\xiaoyao\data\stock_daily_price.parquet")  # 量价数据
auction_df = pd.read_parquet(r"D:\workspace\xiaoyao\data\stock_daily_auction.parquet")    # 竞价数据
industry_df = pd.read_parquet(r"D:\workspace\xiaoyao\data\stock_daily_industry.parquet")  # 行业数据

In [2]:
# 统一日期格式（假设原日期是字符串或带时区的格式）
price_vol_df["date"] = pd.to_datetime(price_vol_df["date"]).dt.date
auction_df["date"] = pd.to_datetime(auction_df["date"]).dt.date
industry_df["date"] = pd.to_datetime(industry_df["date"]).dt.date

# 日期区间过滤
start_date = datetime(2025, 1, 1).date()
end_date = datetime(2025, 9, 25).date()

# 去除停牌的股票 ，仅保留price_vol_df 里 paused==0 的
price_vol_df = price_vol_df[(price_vol_df["date"] >= start_date) & (price_vol_df["date"] <= end_date) & (price_vol_df["paused"] == 0)]
auction_df = auction_df[(auction_df["date"] >= start_date) & (auction_df["date"] <= end_date)]
industry_df = industry_df[(industry_df["date"] >= start_date) & (industry_df["date"] <= end_date)]



In [3]:
# 竞价数据字段重命名（示例：前缀au_）
auction_df = auction_df.rename(columns={
    "volume": "au_volume", 
    "money": "au_money",
})

In [4]:
def calculate_auction_indicators(df):
    """计算竞价相关衍生指标"""
    # 竞价量比 = 竞价成交量 / 最近N日平均竞价成交量（示例N=5，不包含当日，取最近N-1日）
    df["auction_volume_60d_ratio"] = df["au_volume"] / df.groupby("stock_code")["au_volume"].rolling(60, closed="left").mean().reset_index(0, drop=True)
    df["auction_volume_5d_ratio"] = df["au_volume"] / df.groupby("stock_code")["au_volume"].rolling(5, closed="left").mean().reset_index(0, drop=True)
    df["auction_volume_1d_ratio"] = df["au_volume"] / df.groupby("stock_code")["au_volume"].rolling(1, closed="left").mean().reset_index(0, drop=True)


    # 买一量 / 卖一量（假设买一量是a1_v，卖一量是b1_v）
    df["bid1_ask1_ratio"] = df["a1_v"] / df["b1_v"]
    
    # 五档买量和 / 五档卖量和（假设买一至买五量是a1_v~a5_v，卖一至卖五量是b1_v~b5_v）
    df["total_bid_vol"] = df["a1_v"] + df["a2_v"] + df["a3_v"] + df["a4_v"] + df["a5_v"]
    df["total_ask_vol"] = df["b1_v"] + df["b2_v"] + df["b3_v"] + df["b4_v"] + df["b5_v"]
    df["bid_ask_ratio"] = df["total_bid_vol"] / df["total_ask_vol"]
    
    return df

# 计算竞价衍生指标
auction_with_indicators = calculate_auction_indicators(auction_df)

# 仅保留需要的字段 auction_volume_60d_ratio auction_volume_5d_ratio auction_volume_1d_ratio bid1_ask1_ratio bid_ask_ratio
auction_with_indicators = auction_with_indicators[
    [
        "date",
        "stock_code",
        "au_volume",
        "au_money",
        "auction_volume_60d_ratio",
        "auction_volume_5d_ratio",
        "auction_volume_1d_ratio",
        "bid1_ask1_ratio",
        "bid_ask_ratio"
    ]
]
auction_with_indicators

,date,stock_code,au_volume,au_money,auction_volume_60d_ratio,auction_volume_5d_ratio,auction_volume_1d_ratio,bid1_ask1_ratio,bid_ask_ratio
0,2025-01-02,000001.XSHE,583500.0,6844455.00,NaN,NaN,NaN,1.859621,0.903431
1,2025-01-02,000002.XSHE,556110.0,4031797.50,NaN,NaN,NaN,0.588350,0.567159
2,2025-01-02,000004.XSHE,70300.0,965219.00,NaN,NaN,NaN,0.035714,0.383495
3,2025-01-02,000006.XSHE,275200.0,1989696.00,NaN,NaN,NaN,2.744479,0.895698
4,2025-01-02,000007.XSHE,9300.0,65100.00,NaN,NaN,NaN,0.250000,1.620192
...,...,...,...,...,...,...,...,...,...
923315,2025-09-25,688716.XSHG,18973.0,880347.20,0.341929,0.553139,0.977587,1.594896,0.161939
923316,2025-09-25,688726.XSHG,2150.0,100383.50,0.154836,0.295614,0.122396,0.405455,0.660330
923317,2025-09-25,688750.XSHG,10774.0,228301.06,0.415922,1.911572,2.065964,12.162162,1.275258
923318,2025-09-25,688755.XSHG,5980.0,274003.60,0.792986,2.399872,inf,0.147606,0.273996


In [5]:
import pandas as pd
import numpy as np

def calculate_price_vol_indicators(price_vol_df):
    """
    计算无未来函数的技术指标（仅用昨日及之前数据，当日可使用open）
    输入：包含date, stock_code, open, high, low, close, volume的量价数据
    输出：添加10个指标后的DataFrame
    """
    # 复制数据并按股票和日期排序（确保时间顺序正确）
    df = price_vol_df.copy()
    df = df.sort_values(['stock_code', 'date']).reset_index(drop=True)
    grouped = df.groupby('stock_code')
    
    # ----------------------
    # 一、趋势类指标（4个）
    # ----------------------
    # 1. 短期移动平均线（MA5）：基于前5日收盘价（不含当日）
    df['MA5'] = grouped['close'].shift(1).rolling(window=5, min_periods=5).mean().reset_index(0, drop=True)
    
    # 2. 中期移动平均线（MA20）：基于前20日收盘价（不含当日）
    df['MA20'] = grouped['close'].shift(1).rolling(window=20, min_periods=20).mean().reset_index(0, drop=True)
    
    # 3. 平均差率（BIAS5）：(当日开盘价 - 前5日均价) / 前5日均价
    # 注：使用当日open而非close，避免未来数据
    df['BIAS5'] = (df['open'] - df['MA5']) / df['MA5'].replace(0, 0.001) * 100
    
    # 4. 布林带带宽（BOLL_WIDTH）：基于前20日收盘价计算
    df['MB'] = grouped['close'].shift(1).rolling(window=20, min_periods=20).mean().reset_index(0, drop=True)  # 中轨
    df['STD20'] = grouped['close'].shift(1).rolling(window=20, min_periods=20).std().reset_index(0, drop=True)  # 标准差
    df['UB'] = df['MB'] + 2 * df['STD20']  # 上轨
    df['LB'] = df['MB'] - 2 * df['STD20']  # 下轨
    df['BOLL_WIDTH'] = (df['UB'] - df['LB']) / df['MB'].replace(0, 0.001) * 100  # 带宽
    df.drop(columns=['MB', 'STD20', 'UB', 'LB'], inplace=True)
    
    # ----------------------
    # 二、交易量类指标（3个）
    # ----------------------
    # 5. 成交量比（VOL_RATIO）：当日开盘前预估成交量 / 前5日平均成交量
    # 注：若有当日开盘前的预估成交量，可替换为实际数据；此处用昨日成交量模拟
    df['EST_VOLUME'] = grouped['volume'].shift(1)  # 模拟当日预估成交量（实际应使用盘前数据）
    df['VOL_RATIO'] = df['EST_VOLUME'] / grouped['volume'].shift(1).rolling(window=5, min_periods=5).mean().reset_index(0, drop=True)
    df.drop(columns=['EST_VOLUME'], inplace=True)
    
    # 6. 资金流向指数（MFI）：基于前14日数据计算
    df['PREV_CLOSE'] = grouped['close'].shift(1)  # 前日收盘价
    df['TP_PREV'] = (grouped['high'].shift(1) + grouped['low'].shift(1) + grouped['close'].shift(1)) / 3  # 前日典型价格
    df['MF_PREV'] = df['TP_PREV'] * grouped['volume'].shift(1)  # 前日资金流量
    df['POS_MF'] = np.where(grouped['close'].shift(1) > grouped['close'].shift(2), df['MF_PREV'], 0)  # 正资金流量
    df['NEG_MF'] = np.where(grouped['close'].shift(1) < grouped['close'].shift(2), df['MF_PREV'], 0)  # 负资金流量
    df['POS_MF_14'] = grouped['POS_MF'].rolling(window=14, min_periods=14).sum().reset_index(0, drop=True)
    df['NEG_MF_14'] = grouped['NEG_MF'].rolling(window=14, min_periods=14).sum().reset_index(0, drop=True)
    df['MFI'] = 100 - (100 / (1 + df['POS_MF_14'] / df['NEG_MF_14'].replace(0, 0.001)))
    df.drop(columns=['TP_PREV', 'MF_PREV', 'POS_MF', 'NEG_MF', 'POS_MF_14', 'NEG_MF_14'], inplace=True)
    
    # 7. 成交量波动率：基于前10日成交量计算
    df['VOL_MEAN_10'] = grouped['volume'].shift(1).rolling(window=10, min_periods=10).mean().reset_index(0, drop=True)
    df['VOL_STD_10'] = grouped['volume'].shift(1).rolling(window=10, min_periods=10).std().reset_index(0, drop=True)
    df['VOL_VOLATILITY'] = (grouped['volume'].shift(1) - df['VOL_MEAN_10']) / df['VOL_STD_10'].replace(0, 0.001)
    df.drop(columns=['VOL_MEAN_10', 'VOL_STD_10'], inplace=True)
    
    # ----------------------
    # 三、超买超卖类指标（3个）
    # ----------------------
    # 8. 相对强弱指数（RSI14）：基于前14日涨跌计算
    df['CHANGE_PREV'] = grouped['close'].shift(1) - grouped['close'].shift(2)  # 前日涨跌幅
    df['GAIN_PREV'] = np.where(df['CHANGE_PREV'] > 0, df['CHANGE_PREV'], 0)  # 前日上涨幅度
    df['LOSS_PREV'] = np.where(df['CHANGE_PREV'] < 0, -df['CHANGE_PREV'], 0)  # 前日下跌幅度
    df['AVG_GAIN_14'] = grouped['GAIN_PREV'].rolling(window=14, min_periods=14).mean().reset_index(0, drop=True)
    df['AVG_LOSS_14'] = grouped['LOSS_PREV'].rolling(window=14, min_periods=14).mean().reset_index(0, drop=True)
    df['RSI14'] = 100 - (100 / (1 + df['AVG_GAIN_14'] / df['AVG_LOSS_14'].replace(0, 0.001)))
    df.drop(columns=['CHANGE_PREV', 'GAIN_PREV', 'LOSS_PREV', 'AVG_GAIN_14', 'AVG_LOSS_14'], inplace=True)
    
    # 9. 随机指标（KDJ-K值）：基于前9日高低价和前日收盘价
    df['LOW9'] = grouped['low'].shift(1).rolling(window=9, min_periods=9).min().reset_index(0, drop=True)  # 前9日最低价
    df['HIGH9'] = grouped['high'].shift(1).rolling(window=9, min_periods=9).max().reset_index(0, drop=True)  # 前9日最高价
    df['KDJ_K'] = 100 * (grouped['close'].shift(1) - df['LOW9']) / (df['HIGH9'] - df['LOW9']).replace(0, 0.001)
    df.drop(columns=['LOW9', 'HIGH9'], inplace=True)
    
    # 10. 能量潮指标（OBV）：累计前日能量潮
    df['OBV_CHANGE_PREV'] = np.where(
        grouped['close'].shift(1) > grouped['close'].shift(2),
        grouped['volume'].shift(1),
        np.where(grouped['close'].shift(1) < grouped['close'].shift(2), -grouped['volume'].shift(1), 0)
    )
    df['OBV'] = grouped['OBV_CHANGE_PREV'].cumsum().reset_index(0, drop=True)
    df.drop(columns=['OBV_CHANGE_PREV'], inplace=True)
    
    return df

price_vol_with_indicators = calculate_price_vol_indicators(price_vol_df)

In [6]:
# 1. 关联量价（带指标）和竞价（带指标）：按date和stock_code
merged_df = pd.merge(
    price_vol_with_indicators, 
    auction_with_indicators, 
    on=["date", "stock_code"], 
    how="inner"  # 取两者都有的日期和股票
)

# 2. 关联行业数据：按date和stock_code（行业数据可能是每日或历史快照，需确保关联逻辑）
final_df = pd.merge(
    merged_df, 
    industry_df[["date", "stock_code", "zjw_industry_code", "zjw_industry_name"]], 
    on=["date", "stock_code"], 
    how="left"  # 行业数据若缺失，填充为NaN
)

In [7]:
final_df

,date,stock_code,open,close,low,high,volume,money,factor,high_limit,...,OBV,au_volume,au_money,auction_volume_60d_ratio,auction_volume_5d_ratio,auction_volume_1d_ratio,bid1_ask1_ratio,bid_ask_ratio,zjw_industry_code,zjw_industry_name
0,2025-01-02,000001.XSHE,1630.12,1588.43,1582.87,1635.68,1309344.0,2.102923e+09,138.970157,1788.55,...,0.0,583500.0,6844455.00,NaN,NaN,NaN,1.859621,0.903431,J66,货币金融服务
1,2025-01-03,000001.XSHE,1589.82,1581.48,1578.70,1603.72,830884.0,1.320521e+09,138.970157,1746.85,...,0.0,553000.0,6326320.00,NaN,NaN,0.947729,0.403237,0.418685,J66,货币金融服务
2,2025-01-06,000001.XSHE,1581.48,1589.82,1559.25,1595.38,781129.0,1.234306e+09,138.970157,1739.91,...,-830884.0,666200.0,7581356.00,NaN,NaN,1.204702,0.216382,0.150624,J66,货币金融服务
3,2025-01-07,000001.XSHE,1587.04,1599.55,1580.09,1602.33,538146.0,8.583290e+08,138.970157,1748.24,...,-49755.0,444300.0,5073906.00,NaN,NaN,0.666917,3.072398,2.251524,J66,货币金融服务
4,2025-01-08,000001.XSHE,1598.16,1598.16,1584.26,1616.22,764471.0,1.223599e+09,138.970157,1759.36,...,488391.0,308000.0,3542000.00,NaN,NaN,0.693225,0.512500,0.711598,J66,货币金融服务
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923315,2025-09-19,689009.XSHG,70.07,70.13,69.77,72.16,13524265.0,9.577190e+08,1.026022,83.80,...,175957311.0,91397.0,6241501.13,1.134997,1.463785,1.735144,0.826981,1.010372,C39,计算机、通信和其他电子设备制造业
923316,2025-09-22,689009.XSHG,70.80,70.04,69.10,71.42,11957738.0,8.369521e+08,1.026022,84.15,...,189481576.0,84494.0,5830086.00,1.032417,1.200953,0.924472,0.071454,1.494231,C39,计算机、通信和其他电子设备制造业
923317,2025-09-23,689009.XSHG,69.34,68.53,66.28,69.95,16558914.0,1.126597e+09,1.026022,84.04,...,177523838.0,83832.0,5665366.56,1.009803,1.057612,0.992165,1.176471,0.453763,C39,计算机、通信和其他电子设备制造业
923318,2025-09-24,689009.XSHG,67.88,69.14,67.68,69.65,10672516.0,7.332604e+08,1.026022,82.24,...,160964924.0,40995.0,2712229.20,0.487195,0.451748,0.489014,6.315789,3.127779,C39,计算机、通信和其他电子设备制造业


In [8]:
def calculate_future_returns(df):
    """计算未来N日收益率"""
    # 当日收盘价
    df["return_dt"] = (df["close"] - df["open"]) / df["close"] * 100

    # 未来1日收盘价（shift(-1)实现）
    df["close_next1"] = df.groupby("stock_code")["close"].shift(-1)
    df["return_next1"] = (df["close_next1"] - df["open"]) / df["open"] * 100

    # 未来3日收盘价（shift(-3)实现）
    df["close_next3"] = df.groupby("stock_code")["close"].shift(-3)
    df["return_next3"] = (df["close_next3"] - df["open"]) / df["open"] * 100

    # 未来5日收盘价（shift(-5)实现）
    df["close_next5"] = df.groupby("stock_code")["close"].shift(-5)
    df["return_next5"] = (df["close_next5"] - df["open"]) / df["open"] * 100

    return df

final_df = calculate_future_returns(final_df)

In [9]:
# 保存为Parquet（压缩存储，适合大数据）
final_df.to_parquet("wide.parquet", index=False)

# 或保存为CSV（方便非编程工具查看）
# final_df.to_csv("融合后宽表.csv", index=False, encoding="utf-8")

In [10]:
import pandas as pd

def get_top_industries_daily(wide_table_path):
    """
    获取每日auction_volume_60d_ratio前五的行业及其平均收益率
    
    参数:
        wide_table_path: 宽表数据的文件路径（支持parquet或csv格式）
    
    返回:
        top_industries: 包含每日前五行业及其指标的DataFrame
    """
    # 读取宽表数据（根据实际格式选择读取方式）
    if wide_table_path.endswith('.parquet'):
        df = pd.read_parquet(wide_table_path)
    else:
        df = pd.read_csv(wide_table_path, parse_dates=['date'])
    
    # 数据预处理：过滤无效值
    valid_df = df[
        df['zjw_industry_name'].notna() & 
        df['auction_volume_60d_ratio'].notna() &
        df['return_dt'].notna() &
        df['return_next1'].notna()
    ].copy()
    
    # 1. 按日期和行业分组，计算行业平均指标
    industry_daily = valid_df.groupby(['date', 'zjw_industry_name']).agg(
        avg_auction_ratio=('auction_volume_60d_ratio', 'mean'),
        avg_return_dt=('return_dt', 'mean'),
        avg_return_next1=('return_next1', 'mean'),
        stock_count=('stock_code', 'nunique')  # 统计行业内股票数量
    ).reset_index()
    
    # 2. 按日期对行业排序，取每日前五
    # 按日期分组，对平均竞价量比降序排名，即平均竞价量比越大，排名越靠前
    industry_daily['industry_rank'] = industry_daily.groupby('date')['avg_auction_ratio'].rank(
        method='dense',  # 密集排名，相同值排名相同
        ascending=False
    ).astype(int)
    
    # 筛选排名前五的行业
    top_industries = industry_daily[industry_daily['industry_rank'] <= 2].sort_values(
        by=['date', 'industry_rank'], 
        ascending=[False, True]
    ).reset_index(drop=True)
    
    return top_industries

# 示例使用
if __name__ == "__main__":
    # 替换为你的宽表文件路径
    wide_table_path = "wide.parquet"
    
    # 获取结果
    daily_top5 = get_top_industries_daily(wide_table_path)
    
    # 打印最近5天的结果
    print(daily_top5.head(25))  # 每天5条，共显示5天
    
    # 保存结果（可选）
    daily_top5.to_csv("每日前五行业竞价量比及收益率.csv", index=False, encoding='utf-8-sig')
    daily_top5.to_parquet("每日前五行业竞价量比及收益率.parquet", index=False)


          date  zjw_industry_name  avg_auction_ratio  avg_return_dt  \
0   2025-09-24           水的生产和供应业           3.372535       1.117413   
1   2025-09-24              水上运输业           3.183531       0.864927   
2   2025-09-23     电信、广播电视和卫星传输服务           4.279378      -1.056373   
3   2025-09-23            公共设施管理业           3.796313      -2.921733   
4   2025-09-22            公共设施管理业           7.006128      -1.508992   
5   2025-09-22                住宿业           3.441168      -3.392422   
6   2025-09-19                住宿业           3.330854       1.560449   
7   2025-09-19            公共设施管理业           2.767343       2.792758   
8   2025-09-18              管道运输业           8.017895      -4.719101   
9   2025-09-18  木材加工和木、竹、藤、棕、草制品业           6.580438      -2.685879   
10  2025-09-17           纺织服装、服饰业           6.953975      -0.499122   
11  2025-09-17           装卸搬运和仓储业           6.066304      -1.091890   
12  2025-09-16   机动车、电子产品和日用产品修理业           5.324872       8.105960   
13  20